In [94]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import json
import numpy as np

Elimino las filas duplicadas de los dataframes

In [39]:
cast = pd.read_csv('../clean_data/cast.csv',sep='|')
cast.shape[0]

562474

In [40]:
cast.head()

,Unnamed: 0,dict,movie_id,dictionary_format
0,0,"{'cast_id': 14, 'character': 'Woody (voice)', ...",862,1
1,1,"{'cast_id': 15, 'character': 'Buzz Lightyear (...",862,1
2,2,"{'cast_id': 16, 'character': 'Mr. Potato Head ...",862,1
3,3,"{'cast_id': 17, 'character': 'Slinky Dog (voic...",862,1
4,4,"{'cast_id': 18, 'character': 'Rex (voice)', 'c...",862,1


In [41]:
cast = cast.drop_duplicates()
cast.shape[0]

562474

In [42]:
crew = pd.read_csv('../clean_data/crew.csv',sep='|')
crew.shape[0]

464314

In [17]:
crew.head()

,Unnamed: 0,dict,movie_id,dictionary_format
0,0,"{'credit_id': '52fe4284c3a36847f8024f49', 'dep...",862,1
1,1,"{'credit_id': '52fe4284c3a36847f8024f4f', 'dep...",862,1
2,2,"{'credit_id': '52fe4284c3a36847f8024f55', 'dep...",862,1
3,3,"{'credit_id': '52fe4284c3a36847f8024f5b', 'dep...",862,1
4,4,"{'credit_id': '52fe4284c3a36847f8024f61', 'dep...",862,1


In [13]:
crew = crew.drop_duplicates()
crew.shape[0]

464314

---

Elimino la columa Unnamed: 0 y dictionary_format de cada df

In [43]:
cast = cast.drop(columns={'Unnamed: 0','dictionary_format'})
crew = crew.drop(columns={'Unnamed: 0','dictionary_format'})

In [44]:
cast.head()

,dict,movie_id
0,"{'cast_id': 14, 'character': 'Woody (voice)', ...",862
1,"{'cast_id': 15, 'character': 'Buzz Lightyear (...",862
2,"{'cast_id': 16, 'character': 'Mr. Potato Head ...",862
3,"{'cast_id': 17, 'character': 'Slinky Dog (voic...",862
4,"{'cast_id': 18, 'character': 'Rex (voice)', 'c...",862


Transformo todos los valores de 'dict' a formato de diccionario, total se que todos pueden hacerlo ahora con las transformaciones que hice en el notebook 'ETL_credits.ipynb'

In [45]:
cast.dict = cast.dict.apply(lambda x: json.loads(x.replace("'",'"')))

Hago una lista con todas las claves que tienen los diccionarios menos 'profile_path' que no me va a ser util

In [72]:
new_columns = ['cast_id', 'character', 'credit_id', 'gender', 'id', 'name', 'order', 'profile_path']

Creo las nuevas columnas para cada una de las keys

In [91]:
column_fails = []
for k in new_columns:
    try:
        cast[k] = cast.dict.apply(lambda x: x[k])
    except KeyError:
        column_fails.append(k)

In [92]:
column_fails

['credit_id']

In [95]:
cast['credit_id'] = np.nan
indexes_fail = []
for index, dicc in enumerate(cast.dict):
    try:
        cast.at[index, 'credit_id'] = dicc['credit_id']
    except KeyError:
        indexes_fail.append(index)

In [98]:
cast = cast.drop(columns={'profile_path'})

Arreglo manualmente los credit_id ya que son solo tres filas

In [100]:
indexes_fail

[135733, 498861, 537459]

In [134]:
for i in cast.iloc[indexes_fail].dict:
    print(i)

{'cast_id': 5, 'character': 'Clips from For Me and My Gal', 'Easter Parade, & Girl Crazyetc (archive footage), credit_id': '567807ecc3a3681689008d0e', 'gender': 1, 'id': 9066, 'name': 'Judy Garland', 'order': 2, 'profile_path': '/r2p0SpwpkDc5hhg7hqbar9bKMox.jpg'}
{'cast_id': 1, 'character': 'Private detective Kogorô Akechi (2x); A man; Masaki (story Imomushi', 'Kagami jigoku, Kasei no unga, and Mushi), credit_id': '52fe4731c3a36847f81285ab', 'gender': 2, 'id': 13275, 'name': 'Tadanobu Asano', 'order': 0, 'profile_path': '/zlZsST8s1Apm6D6bCyYeWrCKZCy.jpg'}
{'cast_id': 5, 'character': 'Erkki Uolevi Lahti, Dynamiitti-Lahti', 'Tyny, credit_id': '58aac76092514141a0001ddf', 'gender': 2, 'id': 110772, 'name': 'Harri Hyttinen', 'order': 6, 'profile_path': 'None'}


In [ ]:
credits_id_fixed = ['567807ecc3a3681689008d0e','52fe4731c3a36847f81285ab','58aac76092514141a0001ddf']

In [137]:
cast.credit_id.iloc[135733] = '567807ecc3a3681689008d0e'
cast.credit_id.iloc[498861] = '52fe4731c3a36847f81285ab'
cast.credit_id.iloc[537459] = '58aac76092514141a0001ddf'

Y ahora vemos que estan todos los datos guardados efectivamente

In [138]:
cast.count()

dict         562474
movie_id     562474
cast_id      562474
character    562474
gender       562474
id           562474
name         562474
order        562474
credit_id    562474
dtype: int64

Ahora procedo a eliminar la columna 'dict' que contiene los diccionarios ya que ahora son innecesarios

In [ ]:
cast = cast.drop(columns={'dict'})

In [142]:
cast

,movie_id,cast_id,character,gender,id,name,order,credit_id
0,862,14,Woody (voice),2,31,Tom Hanks,0,52fe4284c3a36847f8024f95
1,862,15,Buzz Lightyear (voice),2,12898,Tim Allen,1,52fe4284c3a36847f8024f99
2,862,16,Mr. Potato Head (voice),2,7167,Don Rickles,2,52fe4284c3a36847f8024f9d
3,862,17,Slinky Dog (voice),2,12899,Jim Varney,3,52fe4284c3a36847f8024fa1
4,862,18,Rex (voice),2,12900,Wallace Shawn,4,52fe4284c3a36847f8024fa5
...,...,...,...,...,...,...,...,...
562469,227506,2,,2,544742,Iwan Mosschuchin,0,52fe4ea59251416c7515d7d5
562470,227506,3,,1,1090923,Nathalie Lissenko,1,52fe4ea59251416c7515d7d9
562471,227506,4,,2,1136422,Pavel Pavlov,2,52fe4ea59251416c7515d7dd
562472,227506,5,,0,1261758,Aleksandr Chabrov,3,52fe4ea59251416c7515d7e1


In [143]:
crew

,dict,movie_id
0,"{'credit_id': '52fe4284c3a36847f8024f49', 'dep...",862
1,"{'credit_id': '52fe4284c3a36847f8024f4f', 'dep...",862
2,"{'credit_id': '52fe4284c3a36847f8024f55', 'dep...",862
3,"{'credit_id': '52fe4284c3a36847f8024f5b', 'dep...",862
4,"{'credit_id': '52fe4284c3a36847f8024f61', 'dep...",862
...,...,...
464309,"{'credit_id': '52fe4776c3a368484e0c8399', 'dep...",67758
464310,"{'credit_id': '52fe4776c3a368484e0c839f', 'dep...",67758
464311,"{'credit_id': '533bccebc3a36844cf0011a7', 'dep...",227506
464312,"{'credit_id': '58ebbc26925141281908aa0a', 'dep...",227506


In [144]:
crew.dict = crew.dict.apply(lambda x: json.loads(x.replace("'",'"')))

In [147]:
keys = crew.dict[0].keys()

In [149]:
column_fails = []
for k in keys:
    try:
        crew[k] = crew.dict.apply(lambda x: x[k])
    except KeyError:
        column_fails.append(k)

In [151]:
crew.head()

,dict,movie_id,credit_id,department,gender,id,job,name,profile_path
0,"{'credit_id': '52fe4284c3a36847f8024f49', 'dep...",862,52fe4284c3a36847f8024f49,Directing,2,7879,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg
1,"{'credit_id': '52fe4284c3a36847f8024f4f', 'dep...",862,52fe4284c3a36847f8024f4f,Writing,2,12891,Screenplay,Joss Whedon,/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg
2,"{'credit_id': '52fe4284c3a36847f8024f55', 'dep...",862,52fe4284c3a36847f8024f55,Writing,2,7,Screenplay,Andrew Stanton,/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg
3,"{'credit_id': '52fe4284c3a36847f8024f5b', 'dep...",862,52fe4284c3a36847f8024f5b,Writing,2,12892,Screenplay,Joel Cohen,/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg
4,"{'credit_id': '52fe4284c3a36847f8024f61', 'dep...",862,52fe4284c3a36847f8024f61,Writing,0,12893,Screenplay,Alec Sokolow,/v79vlRYi94BZUQnkkyznbGUZLjT.jpg


Ahora con todas las nuevas columnas creadas, elimino 'profile_path' y 'dict' ya que no las necesito

In [152]:
crew = crew.drop(columns={'dict','profile_path'})
crew

,movie_id,credit_id,department,gender,id,job,name
0,862,52fe4284c3a36847f8024f49,Directing,2,7879,Director,John Lasseter
1,862,52fe4284c3a36847f8024f4f,Writing,2,12891,Screenplay,Joss Whedon
2,862,52fe4284c3a36847f8024f55,Writing,2,7,Screenplay,Andrew Stanton
3,862,52fe4284c3a36847f8024f5b,Writing,2,12892,Screenplay,Joel Cohen
4,862,52fe4284c3a36847f8024f61,Writing,0,12893,Screenplay,Alec Sokolow
...,...,...,...,...,...,...,...
464309,67758,52fe4776c3a368484e0c8399,Sound,0,549356,Original Music Composer,Richard McHugh
464310,67758,52fe4776c3a368484e0c839f,Camera,2,58818,Director of Photography,João Fernandes
464311,227506,533bccebc3a36844cf0011a7,Directing,0,1085341,Director,Yakov Protazanov
464312,227506,58ebbc26925141281908aa0a,Production,2,1195656,Producer,Joseph N. Ermolieff


Finalmente, con toda la data limpia y organizada como corresponde, procedo a exportar los dataframe como archivos 'csv' para su posterior tratamiento

In [153]:
cast.to_csv('../clean_data/final_cast.csv',sep=',',index=False)
crew.to_csv('../clean_data/final_crew.csv',sep=',',index=False)